In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from folium.plugins import MousePosition
from pyproj import Proj, Transformer
import json

In [4]:
# 한글깨짐 방지
plt.rc("font", family = "Malgun GOthic")
plt.rc("axes", unicode_minus=False)
import matplotlib
from matplotlib import font_manager, rc
import warnings
warnings.filterwarnings('ignore')


데이터 정의

- YM : 기준년월
- SIDO : 지역대분류명
- SIGUNGU : 지역중분류명
- FranClass : 소상공인구분
- Type : 업종명
- Time : 시간대
- TotalSpent : 총사용금액
- DisSpent : 재난지원금 사용금액
- NumOfSpent : 총 이용건수
- NumOfDisSpent : 총 재난지원금 이용건수
- POINT_X, POINT_Y : X,Y 좌표

In [12]:
data5 = pd.read_csv('jeju_data5.txt', engine='python', encoding='utf-8')

In [13]:
data6 = pd.read_csv('jeju_data6.txt', engine='python', encoding='utf-8')

In [22]:
data7 = pd.read_csv('jeju_data7.txt', engine='python', encoding='utf-8')

In [15]:
data8 = pd.read_csv('jeju_data8.txt', engine='python', encoding='utf-8')

In [7]:
data5.head()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06
1,2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,877005.7447,1.479816e+06
2,3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,877056.6756,1.479616e+06
3,4,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,877055.9593,1.479766e+06
4,5,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,877055.4817,1.479866e+06


In [16]:
data5.shape, data6.shape, data7.shape, data8.shape

((273183, 14), (281896, 14), (284265, 16), (280085, 14))

In [17]:
data7.head()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,X,Y,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
0,1,1,202007,제주특별자치도,제주시,230600,80000,영세,일반한식,00시,85500,0,4,0,877005.9834,1.479766e+06
1,2,2,202007,제주특별자치도,제주시,230600,80050,영세,단란주점,00시,1960000,0,4,0,877005.7447,1.479816e+06
2,3,3,202007,제주특별자치도,제주시,230650,79850,중소1,편의점,00시,475650,0,43,0,877056.6756,1.479616e+06
3,4,4,202007,제주특별자치도,제주시,230650,80000,영세,편의점,00시,15650,0,1,0,877055.9593,1.479766e+06
4,5,5,202007,제주특별자치도,제주시,230650,80100,영세,주점,00시,82500,0,1,0,877055.4817,1.479866e+06


In [25]:
data7 = data7.drop(['X','Y'], axis=1)

In [26]:
data7

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
0,1,1,202007,제주특별자치도,제주시,영세,일반한식,00시,85500,0,4,0,877005.9834,1.479766e+06
1,2,2,202007,제주특별자치도,제주시,영세,단란주점,00시,1960000,0,4,0,877005.7447,1.479816e+06
2,3,3,202007,제주특별자치도,제주시,중소1,편의점,00시,475650,0,43,0,877056.6756,1.479616e+06
3,4,4,202007,제주특별자치도,제주시,영세,편의점,00시,15650,0,1,0,877055.9593,1.479766e+06
4,5,5,202007,제주특별자치도,제주시,영세,주점,00시,82500,0,1,0,877055.4817,1.479866e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284260,284261,284261,202007,제주특별자치도,제주시,영세,기념품점,x시,6000,0,1,0,950524.6587,1.500760e+06
284261,284262,284262,202007,제주특별자치도,제주시,중소,농축수산품,x시,20000,0,1,0,950524.6587,1.500760e+06
284262,284263,284263,202007,제주특별자치도,제주시,중소,제과점,x시,30000,0,2,0,950524.6587,1.500760e+06
284263,284264,284264,202007,제주특별자치도,제주시,중소1,중국음식,x시,13000,0,1,0,950574.6388,1.500760e+06
